In [1]:
from collections import deque, defaultdict
import os
import logging
import time
import json
import gym
import torch.nn as nn
import torch
import numpy as np

from model import RL_Policy, Semantic_Mapping
from agents.deployment_sem_exp import Sem_Exp_Env_Agent
from utils.storage import GlobalRolloutStorage
from envs import make_vec_envs
from arguments import get_args
import algo
import pickle
import pdb
import faulthandler
import habitat
import envs.utils.pose as pu
import cv2

faulthandler.enable()

config=habitat.get_config("/home/motion/habitat-challenge/configs/challenge_objectnav2022_semantic.local.rgbd.yaml")

env = habitat.Env(
    config=config)


2022-05-28 19:15:49,007 Initializing dataset ObjectNav-v1
2022-05-28 19:15:57,262 initializing sim Sim-v0
[19:15:57:364494]:[Assets] ResourceManager.cpp(345)::loadSemanticSceneDescriptor : SSD File Naming Issue! Neither SceneInstanceAttributes-provided name : /home/motion/habitat-challenge/habitat-challenge-data/data/scene_datasets/hm3d/train/00016-qk9eeNeR4vw/qk9eeNeR4vw.basis.scn  nor constructed filename : /home/motion/habitat-challenge/habitat-challenge-data/data/scene_datasets/hm3d/train/00016-qk9eeNeR4vw/info_semantic.json exist on disk.
2022-05-28 19:15:58,364 Initializing task ObjectNav-v1


[19:15:57:270505]:[Metadata] AttributesManagerBase.h(352)::createFromJsonOrDefaultInternal : <Dataset>: Proposing JSON name : default.scene_dataset_config.json from original name : default| This file does not exist.
[19:15:57:270636]:[Metadata] AssetAttributesManager.cpp(121)::createObject : Asset attributes (capsule3DSolid:capsule3DSolid_hemiRings_4_cylRings_1_segments_12_halfLen_0.75_useTexCoords_false_useTangents_false) created and registered.
[19:15:57:270696]:[Metadata] AssetAttributesManager.cpp(121)::createObject : Asset attributes (capsule3DWireframe:capsule3DWireframe_hemiRings_8_cylRings_1_segments_16_halfLen_1) created and registered.
[19:15:57:270714]:[Metadata] AssetAttributesManager.cpp(121)::createObject : Asset attributes (coneSolid:coneSolid_segments_12_halfLen_1.25_rings_1_useTexCoords_false_useTangents_false_capEnd_true) created and registered.
[19:15:57:270746]:[Metadata] AssetAttributesManager.cpp(121)::createObject : Asset attributes (coneWireframe:coneWireframe_s

[19:15:58:254304]:[Sim] Simulator.cpp(464)::instanceStageForSceneAttributes : 
---
The active scene does not contain semantic annotations : activeSemanticSceneID_ = 0  
---


Assets] ResourceManager.cpp(2186)::loadMaterials : Idx 13:Flat.
[19:15:58:176970]:[Assets] ResourceManager.cpp(2186)::loadMaterials : Idx 14:Flat.
[19:15:58:176975]:[Assets] ResourceManager.cpp(2186)::loadMaterials : Idx 15:Flat.
[19:15:58:176980]:[Assets] ResourceManager.cpp(2186)::loadMaterials : Idx 16:Flat.
[19:15:58:176985]:[Assets] ResourceManager.cpp(2186)::loadMaterials : Idx 17:Flat.
[19:15:58:176990]:[Assets] ResourceManager.cpp(2186)::loadMaterials : Idx 18:Flat.
[19:15:58:176995]:[Assets] ResourceManager.cpp(2186)::loadMaterials : Idx 19:Flat.
[19:15:58:177000]:[Assets] ResourceManager.cpp(2186)::loadMaterials : Idx 20:Flat.
[19:15:58:177005]:[Assets] ResourceManager.cpp(2186)::loadMaterials : Idx 21:Flat.
[19:15:58:177009]:[Assets] ResourceManager.cpp(2186)::loadMaterials : Idx 22:Flat.
[19:15:58:177014]:[Assets] ResourceManager.cpp(2186)::loadMaterials : Idx 23:Flat.
[19:15:58:177019]:[Assets] ResourceManager.cpp(2186)::loadMaterials : Idx 24:Flat.
[19:15:58:177024]:[Asse

In [2]:
class obs_manager:
    def __init__(self):
        self.previous_pose = np.array([0,0,0])

    def refactor_obs(self,obs):
        new_pose = np.array([0,0,0]).astype(float)
        new_pose[:2] = obs['gps']
        new_pose[1] = -new_pose[1]
        new_pose[2] = obs['compass']
        pose_diff = np.array(pu.get_rel_pose_change(new_pose,self.previous_pose))
#         print('This is the new pose: {} and this is the old pose {}'.format(new_pose,self.previous_pose))
        rgb = obs['rgb'].astype(np.uint8)
        depth = obs['depth']
        state = np.concatenate((rgb, depth), axis=2).transpose(2, 0, 1)
        info = {'goal_cat_id':obs['objectgoal'],'sensor_pose':pose_diff}
        self.previous_pose = new_pose
        return state, info
    
def add_category_to_img(img1,category,offset = 0):
    img1 = img1.astype(np.float64)
    font = cv2.FONT_HERSHEY_SIMPLEX
    org = (50+offset,50)
    # fontScale
    fontScale = 1
    # Blue color in BGR
    color = (0, 0, 255)
    # Line thickness of 2 px
    thickness = 2

    # Using cv2.putText() method
    img1 = cv2.putText(img1.astype(np.float64),category, org, font,fontScale, color, thickness, cv2.LINE_AA)
    return img1.astype(np.uint8)

In [3]:
obs = env.reset()
# previous_pose = np.array([0,0,0])
# previous_pose[:2] = obs['gps']
# previous_pose[2] = obs['compass']


om = obs_manager()

In [4]:
args = pickle.load(open('seg_args.pkl','rb'))
args.collision_threshold = 0.1

In [ ]:
class packaged_agent:
    def __init__(self,args,om):
        self.args = args
        np.random.seed(self.args.seed)
        torch.manual_seed(self.args.seed)
        if self.args.cuda:
            torch.cuda.manual_seed(self.args.seed)
        
        self.om = om
        self.num_scenes = 1
        # Setup Logging
        log_dir = "{}/models/{}/".format(args.dump_location, self.args.exp_name)
        dump_dir = "{}/dump/{}/".format(args.dump_location, self.args.exp_name)

        if not os.path.exists(log_dir):
            os.makedirs(log_dir)
        if not os.path.exists(dump_dir):
            os.makedirs(dump_dir)

        logging.basicConfig(
            filename=log_dir + 'train.log',
            level=logging.INFO)
        print("Dumping at {}".format(log_dir))
        # print(args)
        logging.info(self.args)

        # Logging and loss variables
        self.device = self.args.device = torch.device("cuda:0" if self.args.cuda else "cpu")


        self.g_masks = torch.ones(1).float().to(self.device)

        self.best_g_reward = -np.inf

        self.episode_success = []
        self.episode_spl = []
        self.episode_dist = []

        self.g_episode_rewards = []
        self.g_value_losses = []
        self.g_dist_entropies = []
        self.per_step_g_rewards = []
        self.g_process_rewards = []

        # Starting environments
        torch.set_num_threads(8)
        torch.set_grad_enabled(False)

        self.nc = self.args.num_sem_categories + 4  # num channels

        # Calculating full and local map sizes
        self.map_size = self.args.map_size_cm // self.args.map_resolution
        self.full_w, self.full_h = self.map_size, self.map_size
        self.local_w = int(self.full_w / self.args.global_downscaling)
        self.local_h = int(self.full_h / self.args.global_downscaling)

        # Initializing full and local map
        self.full_map = torch.zeros(self.num_scenes, self.nc, self.full_w, self.full_h).float().to(self.device)
        self.local_map = torch.zeros(self.num_scenes, self.nc, self.local_w,
                                self.local_h).float().to(self.device)

        # Initial full and local pose
        self.full_pose = torch.zeros(self.num_scenes, 3).float().to(self.device)
        self.local_pose = torch.zeros(self.num_scenes, 3).float().to(self.device)
        # Origin of local map
        self.origins = np.zeros((self.num_scenes, 3))

        # Local Map Boundaries
        self.lmb = np.zeros((self.num_scenes, 4)).astype(int)

        # Planner pose inputs has 7 dimensions
        # 1-3 store continuous global agent location
        # 4-7 store local map boundaries
        self.planner_pose_inputs = np.zeros((self.num_scenes, 7))
        
        self.init_map_and_pose()
        
        
        
        # Global policy observation space
        self.ngc = 8 + self.args.num_sem_categories
        self.es = 2
        self.g_observation_space = gym.spaces.Box(0, 1,
                                             (self.ngc,
                                              self.local_w,
                                              self.local_h), dtype='uint8')

        # Global policy action space
        self.g_action_space = gym.spaces.Box(low=0.0, high=0.99,
                                        shape=(2,), dtype=np.float32)

        # Global policy recurrent layer size
        self.g_hidden_size = self.args.global_hidden_size

        # Semantic Mapping
        self.sem_map_module = Semantic_Mapping(self.args).to(self.device)
        self.sem_map_module.eval()

        # print('Is the mapping module on cuda? {}'.format(next(sem_map_module.parameters()).is_cuda()))
        # Global policy
        self.g_policy = RL_Policy(self.g_observation_space.shape, self.g_action_space,
                             model_type=1,
                             base_kwargs={'recurrent': self.args.use_recurrent_global,
                                          'hidden_size': self.g_hidden_size,
                                          'num_sem_categories': self.ngc - 8
                                          }).to(self.device)
        self.g_agent = algo.PPO(self.g_policy, self.args.clip_param, self.args.ppo_epoch,
                           self.args.num_mini_batch, self.args.value_loss_coef,
                           self.args.entropy_coef, lr=self.args.lr, eps=self.args.eps,
                           max_grad_norm=self.args.max_grad_norm)

        self.global_input = torch.zeros(self.num_scenes, self.ngc, self.local_w, self.local_h)
        self.global_orientation = torch.zeros(self.num_scenes, 1).long()
        self.intrinsic_rews = torch.zeros(self.num_scenes).to(self.device)
        self.extras = torch.zeros(self.num_scenes, 2)
            # Storage
        self.g_rollouts = GlobalRolloutStorage(self.args.num_global_steps,
                                          self.num_scenes, self.g_observation_space.shape,
                                          self.g_action_space, self.g_policy.rec_state_size,
                                          self.es).to(self.device)

        
        if self.args.load != "0":
            print("Loading model {}".format(self.args.load))
            state_dict = torch.load(self.args.load,
                                    map_location=lambda storage, loc: storage)
            self.g_policy.load_state_dict(state_dict)

            if self.args.eval:
                self.g_policy.eval()
                
        self.poses = torch.from_numpy(np.asarray([0,0,0])).float().to(self.device).view(self.num_scenes,3)
        
        
        self.sem_exp_agent = Sem_Exp_Env_Agent(self.args)
        self.sem_exp_agent.reset()
        self.step = 0
        self.g_reward = torch.from_numpy(np.asarray(
                [0])
            ).float().to(self.device)
        
    def get_local_map_boundaries(self,agent_loc, local_sizes, full_sizes):
            loc_r, loc_c = agent_loc
            local_w, local_h = local_sizes
            full_w, full_h = full_sizes

            if self.args.global_downscaling > 1:
                gx1, gy1 = loc_r - local_w // 2, loc_c - local_h // 2
                gx2, gy2 = gx1 + local_w, gy1 + local_h
                if gx1 < 0:
                    gx1, gx2 = 0, local_w
                if gx2 > full_w:
                    gx1, gx2 = full_w - local_w, full_w

                if gy1 < 0:
                    gy1, gy2 = 0, local_h
                if gy2 > full_h:
                    gy1, gy2 = full_h - local_h, full_h
            else:
                gx1, gx2, gy1, gy2 = 0, full_w, 0, full_h

            return [gx1, gx2, gy1, gy2]

    def init_map_and_pose(self):
        self.full_map.fill_(0.)
        self.full_pose.fill_(0.)
        self.full_pose[:, :2] = self.args.map_size_cm / 100.0 / 2.0

        locs = self.full_pose.cpu().numpy()
        self.planner_pose_inputs[:, :3] = locs
        for e in range(self.num_scenes):
            r, c = locs[e, 1], locs[e, 0]
            loc_r, loc_c = [int(r * 100.0 / self.args.map_resolution),
                            int(c * 100.0 / self.args.map_resolution)]

            self.full_map[e, 2:4, loc_r - 1:loc_r + 2, loc_c - 1:loc_c + 2] = 1.0

            self.lmb[e] = self.get_local_map_boundaries((loc_r, loc_c),
                                              (self.local_w, self.local_h),
                                              (self.full_w, self.full_h))

            self.planner_pose_inputs[e, 3:] = self.lmb[e]
            self.origins[e] = [self.lmb[e][2] * args.map_resolution / 100.0,
                          self.lmb[e][0] * args.map_resolution / 100.0, 0.]

        for e in range(self.num_scenes):
            self.local_map[e] = self.full_map[e, :,
                                    self.lmb[e, 0]:self.lmb[e, 1],
                                    self.lmb[e, 2]:self.lmb[e, 3]]
            self.local_pose[e] = self.full_pose[e] - \
                torch.from_numpy(self.origins[e]).to(self.device).float()
        
    def init_map_and_pose_for_env(self,e):
        self.full_map[e].fill_(0.)
        self.full_pose[e].fill_(0.)
        self.full_pose[e, :2] = args.map_size_cm / 100.0 / 2.0

        self.locs = self.full_pose[e].cpu().numpy()
        self.planner_pose_inputs[e, :3] = self.locs
        r, c = self.locs[1], self.locs[0]
        loc_r, loc_c = [int(r * 100.0 / args.map_resolution),
                        int(c * 100.0 / args.map_resolution)]

        self.full_map[e, 2:4, loc_r - 1:loc_r + 2, loc_c - 1:loc_c + 2] = 1.0

        self.lmb[e] = self.get_local_map_boundaries((loc_r, loc_c),
                                          (self.local_w, self.local_h),
                                          (self.full_w, self.full_h))

        self.planner_pose_inputs[e, 3:] = self.lmb[e]
        self.origins[e] = [self.lmb[e][2] * self.args.map_resolution / 100.0,
                      self.lmb[e][0] * self.args.map_resolution / 100.0, 0.]

        self.local_map[e] = self.full_map[e, :, self.lmb[e, 0]:self.lmb[e, 1], self.lmb[e, 2]:self.lmb[e, 3]]
        self.local_pose[e] = self.full_pose[e] - \
            torch.from_numpy(self.origins[e]).to(self.device).float()
    def update_intrinsic_rew(self,e):
        prev_explored_area = self.full_map[e, 1].sum(1).sum(0)
        self.full_map[e, :, self.lmb[e, 0]:self.lmb[e, 1], self.lmb[e, 2]:self.lmb[e, 3]] = \
            self.local_map[e]
        curr_explored_area = self.full_map[e, 1].sum(1).sum(0)
        self.intrinsic_rews[e] = curr_explored_area - prev_explored_area
        self.intrinsic_rews[e] *= (self.args.map_resolution / 100.)**2  # to m^2
        
    def act(self,obs):
        self.g_step = (self.step // args.num_local_steps) % args.num_global_steps
        self.l_step = self.step % args.num_local_steps
        
        obs,info = self.process_observations(obs)
        if(self.step == 0):
            self.first_map_update(obs,info)
            action = self.get_first_action()
            self.step +=1
            return action
        else:
            self.map_update(obs,info)
            action = self.get_action()
            self.step +=1
            return action
    
    def reset(self):
        self.init_map_and_pose()
        self.step = 0
        
    
    def process_observations(self,obs):
        changed_obs,info = self.om.refactor_obs(obs)
        self.poses = torch.from_numpy(info['sensor_pose']).float().to(self.device).view(1,-1)
#         print(self.poses)
        changed_obs = self.sem_exp_agent._preprocess_obs(changed_obs)
        changed_obs = torch.from_numpy(changed_obs).view(1,*changed_obs.shape).to(self.device)
        return changed_obs,info
    
    def first_map_update(self,obs,info):
        
        _, self.local_map, _, self.local_pose = \
        self.sem_map_module(obs, self.poses, self.local_map, self.local_pose)
        locs = self.local_pose.cpu().numpy()
        self.global_input = torch.zeros(self.num_scenes, self.ngc, self.local_w, self.local_h)
        self.global_orientation = torch.zeros(self.num_scenes, 1).long()
        for e in range(self.num_scenes):
            r, c = locs[e, 1], locs[e, 0]
            loc_r, loc_c = [int(r * 100.0 / self.args.map_resolution),
                            int(c * 100.0 / self.args.map_resolution)]

            self.local_map[e, 2:4, loc_r - 1:loc_r + 2, loc_c - 1:loc_c + 2] = 1.
            self.global_orientation[e] = int((locs[e, 2] + 180.0) / 5.)

        self.global_input[:, 0:4, :, :] = self.local_map[:, 0:4, :, :].detach()
        self.global_input[:, 4:8, :, :] = nn.MaxPool2d(self.args.global_downscaling)(
            self.full_map[:, 0:4, :, :])
        self.global_input[:, 8:, :, :] = self.local_map[:, 4:, :, :].detach()
        self.goal_cat_id = torch.from_numpy(np.asarray(info['goal_cat_id']))
        self.extras = torch.zeros(self.num_scenes, 2)
        self.extras[:, 0] = self.global_orientation[:, 0]
        self.extras[:, 1] = self.goal_cat_id
    
    def map_update(self,obs,info):
        _, self.local_map, _, self.local_pose = \
            self.sem_map_module(obs, self.poses, self.local_map, self.local_pose)

        locs = self.local_pose.cpu().numpy()
        self.planner_pose_inputs[:, :3] = locs + self.origins
        self.local_map[:, 2, :, :].fill_(0.)  # Resetting current location channel
        
        for e in range(self.num_scenes):
            r, c = locs[e, 1], locs[e, 0]
            loc_r, loc_c = [int(r * 100.0 / self.args.map_resolution),
                            int(c * 100.0 / self.args.map_resolution)]
            self.local_map[e, 2:4, loc_r - 2:loc_r + 3, loc_c - 2:loc_c + 3] = 1.
            
        if self.l_step == self.args.num_local_steps - 1:
            # For every global step, update the full and local maps
            for e in range(self.num_scenes):
                self.update_intrinsic_rew(e)

                self.full_map[e, :, self.lmb[e, 0]:self.lmb[e, 1], self.lmb[e, 2]:self.lmb[e, 3]] = \
                    self.local_map[e]
                self.full_pose[e] = self.local_pose[e] + \
                    torch.from_numpy(self.origins[e]).to(self.device).float()

                locs = self.full_pose[e].cpu().numpy()
                r, c = locs[1], locs[0]
                loc_r, loc_c = [int(r * 100.0 / self.args.map_resolution),
                                int(c * 100.0 / self.args.map_resolution)]

                self.lmb[e] = self.get_local_map_boundaries((loc_r, loc_c),
                                                  (self.local_w,self.local_h),
                                                  (self.full_w, self.full_h))

                self.planner_pose_inputs[e, 3:] = self.lmb[e]
                self.origins[e] = [self.lmb[e][2] * self.args.map_resolution / 100.0,
                              self.lmb[e][0] * self.args.map_resolution / 100.0, 0.]

                self.local_map[e] = self.full_map[e, :,
                                        self.lmb[e, 0]:self.lmb[e, 1],
                                        self.lmb[e, 2]:self.lmb[e, 3]]
                self.local_pose[e] = self.full_pose[e] - \
                    torch.from_numpy(self.origins[e]).to(self.device).float()

            locs = self.local_pose.cpu().numpy()
            for e in range(self.num_scenes):
                self.global_orientation[e] = int((locs[e, 2] + 180.0) / 5.)
            self.global_input[:, 0:4, :, :] = self.local_map[:, 0:4, :, :]
            self.global_input[:, 4:8, :, :] = \
                nn.MaxPool2d(self.args.global_downscaling)(
                    self.full_map[:, 0:4, :, :])
            self.global_input[:, 8:, :, :] = self.local_map[:, 4:, :, :].detach()
            self.goal_cat_id = torch.from_numpy(np.asarray(info['goal_cat_id']))
            self.extras[:, 0] = self.global_orientation[:, 0]
            self.extras[:, 1] = self.goal_cat_id
            


        
    def get_first_action(self):
        self.g_rollouts.obs[0].copy_(self.global_input)
        self.g_rollouts.extras[0].copy_(self.extras)

        # Run Global Policy (global_goals = Long-Term Goal)
        self.g_value, self.g_action, self.g_action_log_prob, self.g_rec_states = \
            self.g_policy.act(
                self.g_rollouts.obs[0],
                self.g_rollouts.rec_states[0],
                self.g_rollouts.masks[0],
                extras=self.g_rollouts.extras[0],
                deterministic=False
            )
        cpu_actions = nn.Sigmoid()(self.g_action).cpu().numpy()
        global_goals = [[int(action[0] * self.local_w), int(action[1] * self.local_h)]
                        for action in cpu_actions]
        global_goals = [[min(x, int(self.local_w - 1)), min(y, int(self.local_h - 1))]
                        for x, y in global_goals]

        self.goal_maps = [np.zeros((self.local_w, self.local_h)) for _ in range(self.num_scenes)]

        for e in range(self.num_scenes):
            self.goal_maps[e][global_goals[e][0], global_goals[e][1]] = 1

            planner_inputs = [{} for e in range(self.num_scenes)]
            for e, p_input in enumerate(planner_inputs):
                p_input['map_pred'] = self.local_map[e, 0, :, :].cpu().numpy()
                p_input['exp_pred'] = self.local_map[e, 1, :, :].cpu().numpy()
                p_input['pose_pred'] = self.planner_pose_inputs[e]
                p_input['goal'] = self.goal_maps[e]  # global_goals[e]
                p_input['new_goal'] = 1
                p_input['found_goal'] = 0
                p_input['wait'] = False
                if args.visualize or args.print_images:
                    self.local_map[e, -1, :, :] = 1e-5
                    p_input['sem_map_pred'] = self.local_map[e, 4:, :, :
                                                        ].argmax(0).cpu().numpy()
        action = self.sem_exp_agent.plan_act_and_preprocess(planner_inputs[0])
        return action
    
    
    def get_action(self):
        
        # Add samples to global policy storage
        if self.step == 0:
            self.g_rollouts.obs[0].copy_(self.global_input)
            self.g_rollouts.extras[0].copy_(self.extras)
        else:
            self.g_rollouts.insert(
                self.global_input, self.g_rec_states,
                self.g_action, self.g_action_log_prob, self.g_value,
                self.g_reward, self.g_masks, self.extras
            )
                
        self.g_value, self.g_action, self.g_action_log_prob, self.g_rec_states = \
                self.g_policy.act(
                    self.g_rollouts.obs[self.g_step + 1],
                    self.g_rollouts.rec_states[self.g_step + 1],
                    self.g_rollouts.masks[self.g_step + 1],
                    extras=self.g_rollouts.extras[self.g_step + 1],
                    deterministic=False
                )

        cpu_actions = nn.Sigmoid()(self.g_action).cpu().numpy()
        self.global_goals = [[int(action[0] * self.local_w),
                         int(action[1] * self.local_h)]
                        for action in cpu_actions]
        self.global_goals = [[min(x, int(self.local_w - 1)),
                         min(y, int(self.local_h - 1))]
                        for x, y in self.global_goals]
        g_reward = 0
        g_masks = torch.ones(self.num_scenes).float().to(self.device)
        
        # ------------------------------------------------------------------

        # ------------------------------------------------------------------
        # Update long-term goal if target object is found
        self.found_goal = [0 for _ in range(self.num_scenes)]
        self.goal_maps = [np.zeros((self.local_w, self.local_h)) for _ in range(self.num_scenes)]

        for e in range(self.num_scenes):
#             print('\n\n\n FOUND GOAL!!!')
            self.goal_maps[e][self.global_goals[e][0], self.global_goals[e][1]] = 1

        for e in range(self.num_scenes):
            cn = self.goal_cat_id + 4
            if self.local_map[e, cn, :, :].sum() != 0.:
#                 print('\n\n\nn FOUND GOAL!?')

                self.cat_semantic_map = self.local_map[e, cn, :, :].cpu().numpy()
                self.cat_semantic_scores = self.cat_semantic_map
                self.cat_semantic_scores[self.cat_semantic_scores > 0] = 1.
                self.goal_maps[e] = self.cat_semantic_scores[0]
                self.found_goal[e] = 1
        
        
        planner_inputs = [{} for e in range(self.num_scenes)]
        for e, p_input in enumerate(planner_inputs):
            p_input['map_pred'] = self.local_map[e, 0, :, :].cpu().numpy()
            p_input['exp_pred'] = self.local_map[e, 1, :, :].cpu().numpy()
            p_input['pose_pred'] = self.planner_pose_inputs[e]
            p_input['goal'] = self.goal_maps[e]  # global_goals[e]
            p_input['new_goal'] = self.l_step == self.args.num_local_steps - 1
            p_input['found_goal'] = self.found_goal[e]
            p_input['wait'] = False
#             if args.visualize or args.print_images:
#                 local_map[e, -1, :, :] = 1e-5
#                 p_input['sem_map_pred'] = local_map[e, 4:, :,
#                                                     :].argmax(0).cpu().numpy()
        self.planner_inputs = planner_inputs
        action = self.sem_exp_agent.plan_act_and_preprocess(planner_inputs[0])
        self.train()
        
        return action
    
    
    def train(self):
#         print('called train!')
        if self.g_step % self.args.num_global_steps == self.args.num_global_steps - 1 \
                and l_step == self.args.num_local_steps - 1:
            if not self.args.eval:
                self.g_next_value = self.g_policy.get_value(
                    self.g_rollouts.obs[-1],
                    self.g_rollouts.rec_states[-1],
                    self.g_rollouts.masks[-1],
                    extras=self.g_rollouts.extras[-1]
                ).detach()

                self.g_rollouts.compute_returns(self.g_next_value, self.args.use_gae,
                                           self.args.gamma, self.args.tau)
                g_value_loss, g_action_loss, g_dist_entropy = \
                    self.g_agent.update(g_rollouts)
                self.g_value_losses.append(g_value_loss)
                self.g_action_losses.append(g_action_loss)
                self.g_dist_entropies.append(g_dist_entropy)
#             print('executing g_rollouts_after_update')
            
        self.g_rollouts.after_update()

        torch.set_grad_enabled(False)

p_a = packaged_agent(args,om)

obs = env.reset()
p_a.reset()
img = obs['rgb']
# font

img = add_category_to_img(img,env.current_episode.object_category)

cv2.imshow('agent view',img)

while True:
    for i in range(500):
        action = p_a.act(obs)
    #     print(p_a.step,action,env.current_episode.object_category)
    #     action = None
        keyboard_input = cv2.waitKey(1)
    #     if keyboard_input in (27, ord('q'), ord('Q')):
    #         break
    #     elif keyboard_input == ord('w'):
    #         action = 'MOVE_FORWARD'
    #     elif keyboard_input == ord('a'):
    #         action = 'TURN_LEFT'
    #     elif keyboard_input == ord('d'):
    #         action = 'TURN_RIGHT'
    #     elif keyboard_input == ord('s'):
    #         action = 'STOP'
    #     else:
    #         pass
        if(action is not None):
            obs = env.step(action)
            img = obs['rgb']
        # font

            img = add_category_to_img(img,env.current_episode.object_category)
            img = add_category_to_img(img,str(i),offset = 100)
            img = add_category_to_img(img,action,offset = 200)
            cv2.imshow('agent view',img)
            cv2.imshow('predicted_map',p_a.local_map.cpu().numpy()[0,0,:,:])
            if(env.episode_over):
                metrics = env.get_metrics()
                print('metrics = {}'.format(metrics))
                env.reset()
                p_a.reset()
    #     cv2.waitKey(50)


Dumping at ./tmp//models/exp1/
Loading model pretrained_models/sem_exp.pth
[05/28 19:16:00 detectron2]: Arguments: Namespace(confidence_threshold=0.7, config_file='configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml', input=['input1.jpeg'], opts=['MODEL.WEIGHTS', 'detectron2://COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl', 'MODEL.DEVICE', 'cuda:0'], output=None, video_input=None, webcam=False)


/home/motion/miniconda3/envs/habitat/lib/python3.7/site-packages/torchvision/transforms/transforms.py:258: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "
QObject::moveToThread: Current thread (0x56452f8c3690) is not the object's thread (0x56453220c5c0).
Cannot move to target thread (0x56452f8c3690)

QObject::moveToThread: Current thread (0x56452f8c3690) is not the object's thread (0x56453220c5c0).
Cannot move to target thread (0x56452f8c3690)

QObject::moveToThread: Current thread (0x56452f8c3690) is not the object's thread (0x56453220c5c0).
Cannot move to target thread (0x56452f8c3690)

QObject::moveToThread: Current thread (0x56452f8c3690) is not the object's thread (0x56453220c5c0).
Cannot move to target thread (0x56452f8c3690)

QObject::moveToThread: Current thread (0x56452f8c3690) is not the object's thread (0x56453220c5c0).
C

/home/motion/miniconda3/envs/habitat/lib/python3.7/site-packages/torch/nn/functional.py:3891: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  "Default grid_sample and affine_grid behavior has changed "
QObject::moveToThread: Current thread (0x56452f8c3690) is not the object's thread (0x56453220c5c0).
Cannot move to target thread (0x56452f8c3690)

QObject::moveToThread: Current thread (0x56452f8c3690) is not the object's thread (0x56453220c5c0).
Cannot move to target thread (0x56452f8c3690)

QObject::moveToThread: Current thread (0x56452f8c3690) is not the object's thread (0x56453220c5c0).
Cannot move to target thread (0x56452f8c3690)

QObject::moveToThread: Current thread (0x56452f8c3690) is not the object's thread (0x56453220c5c0).
Cannot move to target thread (0x56452f8c3690)

QObject::moveToThread: Current thr

metrics = {'distance_to_goal': 15.657463073730469, 'success': 0.0, 'spl': 0.0, 'softspl': 0.0}
metrics = {'distance_to_goal': 5.853246212005615, 'success': 0.0, 'spl': 0.0, 'softspl': 0.4700397981314022}
metrics = {'distance_to_goal': 9.500822067260742, 'success': 0.0, 'spl': 0.0, 'softspl': 0.0}


In [ ]:
p_a.planner_inputs[0]['goal'][0]

In [ ]:
from matplotlib import pyplot as plt
plt.imshow(p_a.local_map[0,0,:,:].cpu().numpy())

In [ ]:


# Storage
g_rollouts = GlobalRolloutStorage(args.num_global_steps,
                                  num_scenes, g_observation_space.shape,
                                  g_action_space, g_policy.rec_state_size,
                                  es).to(device)

In [ ]:
print("Loading model {}".format(args.load))
state_dict = torch.load(args.load,
                        map_location=lambda storage, loc: storage)
g_policy.load_state_dict(state_dict)
if args.eval:
    g_policy.eval()

In [ ]:
poses = torch.from_numpy(np.array([[0,0,0]])).float().to(device)

sem_exp_agent = Sem_Exp_Env_Agent(args)

In [ ]:
obs = env.reset()

changed_obs,info = om.refactor_obs(obs)
poses = torch.from_numpy(info['sensor_pose']).float().to(device).view(1,-1)

changed_obs = sem_exp_agent._preprocess_obs(changed_obs)
changed_obs = torch.from_numpy(changed_obs).view(1,*changed_obs.shape).to(device)
_, local_map, _, local_pose = \
        sem_map_module(changed_obs, poses, local_map, local_pose)

In [ ]:
locs = local_pose.cpu().numpy()
global_input = torch.zeros(num_scenes, ngc, local_w, local_h)
global_orientation = torch.zeros(num_scenes, 1).long()
for e in range(num_scenes):
    r, c = locs[e, 1], locs[e, 0]
    loc_r, loc_c = [int(r * 100.0 / args.map_resolution),
                    int(c * 100.0 / args.map_resolution)]

    local_map[e, 2:4, loc_r - 1:loc_r + 2, loc_c - 1:loc_c + 2] = 1.
    global_orientation[e] = int((locs[e, 2] + 180.0) / 5.)
    
global_input[:, 0:4, :, :] = local_map[:, 0:4, :, :].detach()
global_input[:, 4:8, :, :] = nn.MaxPool2d(args.global_downscaling)(
    full_map[:, 0:4, :, :])
global_input[:, 8:, :, :] = local_map[:, 4:, :, :].detach()
goal_cat_id = torch.from_numpy(np.asarray(info['goal_cat_id']))
extras = torch.zeros(num_scenes, 2)
extras[:, 0] = global_orientation[:, 0]
extras[:, 1] = goal_cat_id

In [ ]:
g_rollouts.obs[0].copy_(global_input)
g_rollouts.extras[0].copy_(extras)

# Run Global Policy (global_goals = Long-Term Goal)
g_value, g_action, g_action_log_prob, g_rec_states = \
    g_policy.act(
        g_rollouts.obs[0],
        g_rollouts.rec_states[0],
        g_rollouts.masks[0],
        extras=g_rollouts.extras[0],
        deterministic=False
    )

In [ ]:
cpu_actions = nn.Sigmoid()(g_action).cpu().numpy()
global_goals = [[int(action[0] * local_w), int(action[1] * local_h)]
                for action in cpu_actions]
global_goals = [[min(x, int(local_w - 1)), min(y, int(local_h - 1))]
                for x, y in global_goals]

goal_maps = [np.zeros((local_w, local_h)) for _ in range(num_scenes)]

for e in range(num_scenes):
    goal_maps[e][global_goals[e][0], global_goals[e][1]] = 1
    
    planner_inputs = [{} for e in range(num_scenes)]
    for e, p_input in enumerate(planner_inputs):
        p_input['map_pred'] = local_map[e, 0, :, :].cpu().numpy()
        p_input['exp_pred'] = local_map[e, 1, :, :].cpu().numpy()
        p_input['pose_pred'] = planner_pose_inputs[e]
        p_input['goal'] = goal_maps[e]  # global_goals[e]
        p_input['new_goal'] = 1
        p_input['found_goal'] = 0
        p_input['wait'] = False
        if args.visualize or args.print_images:
            local_map[e, -1, :, :] = 1e-5
            p_input['sem_map_pred'] = local_map[e, 4:, :, :
                                                ].argmax(0).cpu().numpy()

In [ ]:
from matplotlib import pyplot as plt
plt.imshow(local_map.cpu().numpy().reshape(local_map.shape[1:])[0,:,:])